<a href="https://colab.research.google.com/github/abhishekDey11209/Projects/blob/main/Content_based_recommendation_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#importing Libraries
import pandas as pd
import numpy as np
import difflib      #used to find the close match given by the user
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre Processing

In [198]:
#loading the data in a pandas dataframe
movies_df = pd.read_csv('movies.csv')

#viewing all the rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

movies_df.head(1)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [3]:
#checking the rows and columns of the dataframe
movies_df.shape

(4803, 24)

In [199]:
#checking the name of the coulmns
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [200]:
#selecting features that we need
features = ['genres', 'keywords','overview','tagline', 'title', 'cast', 'director']

In [201]:
#handle missing values (null values replacing with empty string) in a DataFrame using pandas
for i in features:
  movies_df[i] = movies_df[i].fillna('')


In [202]:
movies_df.head(1)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [203]:
movies_df.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [204]:
movies = movies_df[['index', 'title', 'genres', 'keywords','overview','tagline', 'cast', 'director']]

In [206]:
movies.head(1)

,index,title,genres,keywords,overview,tagline,cast,director
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron


In [207]:
#creating a copy of original dataframe movies and removing the duplicates
movies1 = movies.drop_duplicates(subset='title')

# Reset the index of 'movies1' to start from 0
movies1.reset_index(drop=True, inplace=True)
movies1.shape

(4800, 8)

Feature extraction


In [208]:
#concating the features into new a new columns
movies1['combined_features'] = movies1['title']+ ' ' + movies1['genres']+ ' ' + movies1['keywords']+ ' ' + movies1['overview']+ ' ' + movies1['tagline']+ ' ' + movies1['cast']+ ' ' + movies1['director']

<ipython-input-208-faedb239f4b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies1['combined_features'] = movies1['title']+ ' ' + movies1['genres']+ ' ' + movies1['keywords']+ ' ' + movies1['overview']+ ' ' + movies1['tagline']+ ' ' + movies1['cast']+ ' ' + movies1['director']


In [209]:
movies1.head(1)

,index,title,genres,keywords,overview,tagline,cast,director,combined_features
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Avatar Action Adventure Fantasy Science Fictio...


In [89]:
##Alternative way to concat the features

#concatinating all the featurs and adding a new column in movies dataframe
# List of columns to concatenate

#columns_to_concat = ['index', 'title', 'genres', 'keywords', 'overview', 'tagline', 'cast', 'director']



# Create a new column 'combined_text' containing the concatenated text


#movies1['combined_text'] = movies1[columns_to_concat].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [216]:
#converting text data to feature vector
vectorization = TfidfVectorizer()

text_vectorization = vectorization.fit_transform(movies1['combined_features'])

In [ ]:
print(text_vectorization)

In [212]:
#getting the similarty score using cosine_similarity
cos = cosine_similarity(text_vectorization)

In [213]:
print(cos)

[[1.         0.05401348 0.03085241 ... 0.02435463 0.02809774 0.00665853]
 [0.05401348 1.         0.04099888 ... 0.05355352 0.03667816 0.01552801]
 [0.03085241 0.04099888 1.         ... 0.0227007  0.04256264 0.01229254]
 ...
 [0.02435463 0.05355352 0.0227007  ... 1.         0.02988431 0.03920881]
 [0.02809774 0.03667816 0.04256264 ... 0.02988431 1.         0.02880436]
 [0.00665853 0.01552801 0.01229254 ... 0.03920881 0.02880436 1.        ]]


In [167]:
movie_name = input('entre the name of the movie : ')

entre the name of the movie : avatar


In [168]:
#finding the close match given by the user

movie_titles = movies1['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name,movie_titles)

In [169]:
find_close_match

['Avatar']

In [170]:
#only taking the first value from the list
close_match = find_close_match[0]

In [171]:
#find the index of the movie
dummy_value = movies1[movies1['title'] == close_match]
dummby_value_index  = dummy_value['index'].values[0]

In [172]:
dummby_value_index

0

In [173]:
#getting the similarity score of that perticualr movies
cos_score = list(enumerate(cos[dummby_value_index]))

In [174]:
cos_score.sort(key=lambda x: x[1], reverse=True)

In [175]:
#take top 20 values with higest similarity score
top_20_indexes = cos_score[:20]

In [ ]:
top_20_indexes

In [177]:
#extracting the indexes from the list
new_top_20_indexes = []
for index, score in top_20_indexes:
    new_top_20_indexes.append(index)

In [ ]:
new_top_20_indexes

In [ ]:
#prining top 20 recommended movies

print("Suggested movies: \n")

for i in new_top_20_indexes:
  recommended_movies = movies[movies['index'] == i]
  print(recommended_movies['title'].values[0])

Incorporating all the codes in a single cell

In [225]:
movie_name = input('Entre the name of the movie : ')
movie_titles = movies1['title'].tolist()
#finding the closest values from the list
find_close_match = difflib.get_close_matches(movie_name,movie_titles)

#only taking the first value from the list
close_match = find_close_match[0]

#find the index of the movie
dummy_value = movies1[movies1['title'] == close_match]
dummby_value_index  = dummy_value['index'].values[0]

#getting the similarity score of that perticualr movies
cos_score = list(enumerate(cos[dummby_value_index]))

#sorting the similarity score in descending order
cos_score.sort(key=lambda x: x[1], reverse=True)

#take top 20 values
top_20_indexes = cos_score[:20]

#extracting the indexes from the list
new_top_20_indexes = []
for index, score in top_20_indexes:
    new_top_20_indexes.append(index)

#prining top 20 recommended movies

print("Suggested movies: \n")

for i in new_top_20_indexes:
  recommended_movies = movies[movies['index'] == i]
  title = recommended_movies['title'].values[0]
  genres = recommended_movies['genres'].values[0]

    # Print the information
  print(f"Title: {title}")
  print(f"Genres: {genres}")
  print("\n")

Entre the name of the movie : Avengers
Suggested movies: 

Title: The Avengers
Genres: Science Fiction Action Adventure


Title: Avengers: Age of Ultron
Genres: Action Adventure Science Fiction


Title: Captain America: The Winter Soldier
Genres: Action Adventure Science Fiction


Title: Captain America: Civil War
Genres: Adventure Action Science Fiction


Title: Iron Man 2
Genres: Adventure Action Science Fiction


Title: X-Men
Genres: Adventure Action Science Fiction


Title: The Incredible Hulk
Genres: Science Fiction Action Adventure


Title: Thor: The Dark World
Genres: Action Adventure Fantasy


Title: X-Men: Apocalypse
Genres: Science Fiction


Title: Captain America: The First Avenger
Genres: Action Adventure Science Fiction


Title: Iron Man 3
Genres: Action Adventure Science Fiction


Title: X-Men: Days of Future Past
Genres: Action Adventure Fantasy Science Fiction


Title: Ant-Man
Genres: Science Fiction Action Adventure


Title: X-Men: The Last Stand
Genres: Adventure Acti